# MSA 2023 Phase 2 - Part 3

In [1]:
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


from sklearn.model_selection import train_test_split
import numpy as np
import requests, json, os

### Connect to workspace

In [2]:
ws = Workspace.from_config(path="config.json") 
print(ws)

Workspace.create(name='MSA2023-Phase2-Azure', subscription_id='2b48bec8-2465-458d-98ce-3445ace62d9f', resource_group='MSA2023-Phase2-Azure')


### Register model

In [3]:
from sklearn.linear_model import LinearRegression
lr_model = Model.register(ws, model_name = "examlrmodel", model_path = "lr_model.pkl")

Registering model linear_regression_exam


### Create entry script

Registering a model only uploads it onto the cloud, there is no code that can allow us to interface with it. As such, an entry script needs to be created that will run when the model receives data. This script contains two functions:

1. ```init()```: loads the model via a global variable
2. ```run()```: loads the input data, runs the model on it, and returns its predictions to the client

For the Iris dataset, a script called echo_score.py has been written below.

In [4]:
# echo_score.py

# Since the model works with label-encoded data, we can create a dictionary to get the actual class names

def init():
    global model
    model =  LinearRegression()
    model.load_model(os.path.join(os.getenv("AZUREML_MODEL_DIR"), "lr_model.pkl"))

def run(request):
    data = json.loads(request)
    data = np.array(data["data"])
    response = model.predict(data)
    return [classes.get(key) for key in response]

### Create remote virtual environment

After registering our model and creating an entry script, we need to create a remote virtual environment. This environment should have exactly the same modules (including their versions) as the virtual environment on our local machine where we know our model runs.

Under the hood, Azure is creating a Docker container which is essentially a more isolated version of a virtual machine (you don't need to know the specifics of Docker to complete this part). This container will contain the remote virtual environment we want our model to run inside.

There are two options to create a remote virtual environment, as shown below. Using the first option is the easiest but may result in deployment errors, in which case try the second option.

In [5]:
# OPTION 1: Create from Pip requirements
env = Environment.from_pip_requirements(name="examlrmodel", file_path="requirements.txt")

### Create inference configuration

An inference configuration contains all the configuration settings needed for your remote virtual environment.

In [6]:
inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir",
    entry_script="echo_score.py",
)

### Deploy model

In [7]:
# Change the amount of CPU or memory depending on what kind of model used
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(
    ws,
    "examlrmodel",
    [lr_model],
    inference_config,
    aci_config,
    overwrite=True,
)

# Once the model has been deployed, it will appear in the Endpoints section of the Azure Machine Learning Studio
service.wait_for_deployment(show_output=True)

C:\Users\kimby\AppData\Local\Temp\ipykernel_20872\3608784840.py:3: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-08-08 20:21:05+12:00 Creating Container Registry if not exists.
2023-08-08 20:21:05+12:00 Registering the environment.
2023-08-08 20:21:07+12:00 Building image..
2023-08-08 20:31:22+12:00 Generating deployment configuration..
2023-08-08 20:31:23+12:00 Submitting deployment to compute.
2023-08-08 20:31:26+12:00 Checking the status of deployment examlrmodel..
2023-08-08 20:37:42+12:00 Checking the status of inference endpoint examlrmodel.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 8d9b7e4f-8804-4424-b22f-8b907883ffc0
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: examlrmodel. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 3d66ed74acc14ee899a2aeda0467c86a.azurecr.io/azureml/azureml_09d718ac3b59421fde430abe1c93c215 locally. Please refer to https://aka.ms/debugimage#service-launch-fails 

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 8d9b7e4f-8804-4424-b22f-8b907883ffc0
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: examlrmodel. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 3d66ed74acc14ee899a2aeda0467c86a.azurecr.io/azureml/azureml_09d718ac3b59421fde430abe1c93c215 locally. Please refe

### Test model endpoint

Our deployed model can be called via a REST API. If it returns a prediction, then we have successfully deployed our model!

In [ ]:
uri = service.scoring_uri

# Your API endpoint will look like this: http://16a75b9c-e6cc-47cd-89f6-215e077c43a9.australiaeast.azurecontainer.io/score
requests.get("<Add your API endpoint here>")
headers = {"Content-Type": "application/json"}
data = {
    "data": [[6.1, 2.8, 4.7, 1.2]],
}
data = json.dumps(data)

# Output a classification/regression result back to the user
response = requests.post(uri, data=data, headers=headers)
print(response.json())

NameError: name 'service' is not defined